In [1]:
from predictor import fast_predict,generate_test_data
from trainer import generate_train_data,Model,train
from torch.utils.data import  DataLoader
from torch import nn,optim
import torch
import os

In [2]:
num_classes = 32
batch_size = 2048
input_size = 1
model_dir = 'model'
window_size = 10
num_layers = 2
hidden_size = 64
file_dir = 'data_dev'
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
version='v0.1'

In [3]:
model = Model(input_size, hidden_size, num_layers, num_classes).to(device)

In [4]:
train_dataset = generate_train_data(file_dir + '/hdfs_train')
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
# writer = Summary# writer(log_dir='log/' + log)

loading data: 4500it [00:00, 7097.77it/s]


Number of sessions(data_dev/hdfs_train): 4500
Number of seqs(data_dev/hdfs_train): 92639


In [5]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=31)
optimizer = optim.Adam(model.parameters())

In [6]:
model_name = 'data_dir={}_version={}'.format(file_dir, version)
if os.path.exists(model_dir + '/' + model_name + '.pt'):
    model.load_state_dict(torch.load(model_dir + '/' + model_name + '.pt'))

In [7]:
# Train the model
model.train()
train(model, dataloader, criterion, optimizer, current_epoch=0, num_epochs=100)

Epoch [1/100], train_loss: 2.7824
Epoch [2/100], train_loss: 1.8378
Epoch [3/100], train_loss: 1.5816
Epoch [4/100], train_loss: 1.4542
Epoch [5/100], train_loss: 1.3445
Epoch [6/100], train_loss: 1.2220
Epoch [7/100], train_loss: 1.0952
Epoch [8/100], train_loss: 0.9770
Epoch [9/100], train_loss: 0.8963
Epoch [10/100], train_loss: 0.8420
Epoch [11/100], train_loss: 0.7983
Epoch [12/100], train_loss: 0.7629
Epoch [13/100], train_loss: 0.7338
Epoch [14/100], train_loss: 0.7103
Epoch [15/100], train_loss: 0.6911
Epoch [16/100], train_loss: 0.6747
Epoch [17/100], train_loss: 0.6595
Epoch [18/100], train_loss: 0.6470
Epoch [19/100], train_loss: 0.6347
Epoch [20/100], train_loss: 0.6229
Epoch [21/100], train_loss: 0.6145
Epoch [22/100], train_loss: 0.6053
Epoch [23/100], train_loss: 0.5976
Epoch [24/100], train_loss: 0.5909
Epoch [25/100], train_loss: 0.5855
Epoch [26/100], train_loss: 0.5782
Epoch [27/100], train_loss: 0.5736
Epoch [28/100], train_loss: 0.5685
Epoch [29/100], train_loss: 0

In [10]:
model_name

'data_dir=data_dev_version=v0.1'

In [9]:
model_name = 'data_dir={}_version={}'.format(file_dir, version)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
torch.save(model.state_dict(), model_dir + '/' + model_name + '.pt')
# writer.close()
print('Finished Training')

Finished Training


In [15]:
model.eval()
batch_size = 20000
test_normal_session, test_normal_dataset, test_normal_seq, test_normal_label = generate_test_data(
    file_dir+'/hdfs_test_normal', window_size)
normal_dataloader = DataLoader(test_normal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_abnormal_session, test_abnormal_dataset, test_abnormal_seq, test_abnormal_label = generate_test_data(
    file_dir+'/hdfs_test_abnormal', window_size)
abnormal_dataloader = DataLoader(test_abnormal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

normal:: 100%|████████████████████████████████████████████████████████████████| 14177/14177 [00:00<00:00, 15227.73it/s]


Number of sessions(data/hdfs_test_normal): 14177
Number of seqs(data/hdfs_test_normal): 269570


normal:: 100%|██████████████████████████████████████████████████████████████████| 4123/4123 [00:00<00:00, 15214.00it/s]


Number of sessions(data/hdfs_test_abnormal): 4123
Number of seqs(data/hdfs_test_abnormal): 88410


In [19]:
model.eval()
test_normal_result, test_abnormal_result = fast_predict(model, normal_dataloader, abnormal_dataloader,
                                                        test_normal_session,test_abnormal_session, 
                                                        10,window_size)

normal: 14it [00:40,  2.91s/it]
abnormal: 5it [00:15,  3.15s/it]

elapsed_time: 56.677s
false positive (FP): 393, false negative (FN): 1581, Precision: 86.610%, Recall: 61.654%, F1-measure: 72.032%
Finished Predicting


In [4]:
# model_name = 'softmax_data_dir=data_official_version=v0.0'
model_name = 'add_padding_batch_size=2048_epoch=300_window_size=10'
# 'data_dir={}_version={}'.format(file_dir, 'v0.0')
# if os.path.exists(model_dir + '/' + model_name + '.pt'):
model.load_state_dict(torch.load(model_dir + '/' + model_name + '.pt'))
model.eval()

Model(
  (lstm): LSTM(1, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=32, bias=True)
)

In [8]:
file_dir='./data_official'
batch_size = 20000
test_normal_session, test_normal_dataset, test_normal_seq, test_normal_label = generate_test_data(
    file_dir+'/hdfs_test_normal', window_size)
normal_dataloader = DataLoader(test_normal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_abnormal_session, test_abnormal_dataset, test_abnormal_seq, test_abnormal_label = generate_test_data(
    file_dir+'/hdfs_test_abnormal', window_size)
abnormal_dataloader = DataLoader(test_abnormal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

normal:: 100%|████████████████████████████████████████████████████████████████| 14134/14134 [00:00<00:00, 18672.75it/s]


Number of sessions(./data_official/hdfs_test_normal): 14134
Number of seqs(./data_official/hdfs_test_normal): 268765


normal:: 100%|██████████████████████████████████████████████████████████████████| 4123/4123 [00:00<00:00, 21700.13it/s]

Number of sessions(./data_official/hdfs_test_abnormal): 4123
Number of seqs(./data_official/hdfs_test_abnormal): 88410


In [11]:
test_normal_result, test_abnormal_result = fast_predict(model, normal_dataloader, abnormal_dataloader,
            test_normal_session,test_abnormal_session, 
            5,window_size)

normal: 14it [00:38,  2.74s/it]
abnormal: 5it [00:12,  2.54s/it]

elapsed_time: 51.116s
false positive (FP): 1309, false negative (FN): 1030, Precision: 70.264%, Recall: 75.018%, F1-measure: 72.563%
Finished Predicting
